In [1]:
from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import numpy as np
import re
from datetime import datetime
import nltk
import pickle
from nltk.corpus import stopwords
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer

In [2]:
consumer = KafkaConsumer(
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m)
)
consumer.subscribe('data_to_predict')

In [ ]:
# pip install tensorflow


In [3]:
import tensorflow as tf 
load_model = tf.keras.models.load_model('cnn_model_without_duration.h5')
print(load_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 36, 36, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

In [4]:
from kafka import KafkaConsumer
from kafka import KafkaProducer

In [5]:
#Use when using KafkaProducer to produce decision

import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [6]:
producer2 = KafkaProducer(bootstrap_servers='localhost:9092',value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8'))


In [ ]:
# from confluent_kafka import SerializingProducer
# from confluent_kafka.schema_registry.json_schema import JSONSerializer
# from confluent_kafka.schema_registry import SchemaRegistryClient

In [ ]:
# schema_registry_client = SchemaRegistryClient({
#         'url': 'http://localhost:8085/',
# })

In [ ]:
# from uuid import uuid4


In [ ]:
# class comment_data(object):
#     """
#     User record
#     Args:
#         name (str): User's name
#         favorite_number (int): User's favorite number
#         favorite_color (str): User's favorite color
#         address(str): User's address; confidential
#     """
#     def __init__(self, spanId, movieId, userId, comment,status):
#         self.spanId = spanId
#         self.movieId = movieId
#         self.userId = userId
#         self.comment = comment
#         self.status = status

# #         # address should not be serialized, see user_to_dict()
# #         self._address = address

In [ ]:
# def data_to_dict(comment_data, ctx):
#     """
#     Returns a dict representation of a User instance for serialization.
#     Args:
#         user (User): User instance.
#         ctx (SerializationContext): Metadata pertaining to the serialization
#             operation.
#     Returns:
#         dict: Dict populated with user attributes to be serialized.
#     """
#     # User._address must not be serialized; omit from dict
#     return dict(spanId=comment_data.spanId,
#                 movieId=comment_data.movieId,
#                 userId=comment_data.userId,
#                comment=comment_data.comment,
#                status=comment_data.status)

In [ ]:
#  def delivery_report(err, msg):
#     """
#     Reports the failure or success of a message delivery.
#     Args:
#         err (KafkaError): The error that occurred on None on success.
#         msg (Message): The message that was produced or failed.
#     Note:
#         In the delivery report callback the Message.key() and Message.value()
#         will be the binary format as encoded by any configured Serializers and
#         not the same object that was passed to produce().
#         If you wish to pass the original object(s) for key and value to delivery
#         report callback we recommend a bound callback or lambda where you pass
#         the objects along.
#     """
#     if err is not None:
#         print("Delivery failed for User record {}: {}".format(msg.key(), err))
#         return
#     print('User record {} successfully produced to {} [{}] at offset {}'.format(
#         msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [ ]:
#     schema_str = """
#     {
#      "$schema": "http://json-schema.org/draft-07/schema#",
#       "title": "Data_to_predict",
#       "description": "data to predict using ml model",
#       "type": "object",
#       "properties": {
#         "spanId": {
#           "description": "sapn id",
#           "type": "string"
#         },
#         "movieId": {
#           "description": "id of the movie",
#           "type": "string"
#         },
#         "userId": {
#           "description": "User's id",
#           "type": "string"
#         },
#         "comment": {
#           "description": "comemnt made by new user",
#           "type": "string"
#         },
#         "status": {
#           "description": "tells whether a new_commnent is normal or malicious",
#           "type": "string"
#         }        

#       },
#       "required": [ "spanId", "movieId", "userId","status" ]
#     }
#     """

In [ ]:
# json_serializer = JSONSerializer(schema_str, schema_registry_client, data_to_dict)

In [ ]:
# from confluent_kafka.serialization import StringSerializer

In [ ]:
# producer_conf = {'bootstrap.servers': 'localhost:9092',
#                  'key.serializer': StringSerializer('utf_8'),
#                      'value.serializer':json_serializer}

In [ ]:
# producer = SerializingProducer(producer_conf)

In [ ]:
# while True:
#     # Serve on_delivery callbacks from previous calls to produce()
#     producer.poll(0.0)
#     try:
#         new_comment = comment_data(spanId=user_name,
#                         movieId=user_address,
#                         userId=user_favorite_color,
#                         comment=user_favorite_number,
#                                   status)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
#         producer.produce(topic='test_predict', value=new_comment,
#                              on_delivery=delivery_report)
#     except KeyboardInterrupt:
#         break
#     except ValueError:
#         print("Invalid input, discarding record...")
#         continue

# print("\nFlushing records...")
# producer.flush()

In [ ]:
# producer_conf['bootstrap.servers'] = ['broker.1']
# producer_conf['value.serializer'] = json_serializer

In [ ]:
# producer = SerializingProducer(producer_conf)

In [ ]:
# producer = KafkaProducer(bootstrap_servers='localhost:9092')
#value_serializer=lambda v: json.dumps(v, cls=NumpyArrayEncoder).encode('utf-8')

In [ ]:
# !python -V

In [ ]:
# conda install python=3.7.6

In [ ]:
# from kafka_schema_registry import prepare_producer

# # SAMPLE_SCHEMA = {
# #     "type": "record",
# #     "name": "TestType",
# #     "fields" : [
# #         {"name": "age", "type": "int"},
# #         {"name": "name", "type": ["null", "string"]}
# #     ]
# # }



# SAMPLE_SCHEMA = {
#   "type": "object",
#   "additionalProperties": True,
#   "description": "Sample schema to help you get started.",
#   "properties": {
#     "spanId": {
#       "description": "The integer type is used for integral numbers.",
#       "type": "integer"
#     },
#     "movieId": {
#       "description": "The number type is used for any numeric type, either integers or floating point numbers.",
#       "type": "number"
#     },
#     "userId": {
#       "description": "The string type is used for strings of text.",
#       "type": "number"
#     },
#     "comemnt": {
#       "description": "The string type is used for strings of text.",
#       "type": "string"
#     },
#     "status": {
#       "description": "The string type is used for strings of text.",
#       "type": "string"
#     }
#   }
# }

# producer = prepare_producer(
#         ['localhost:9092'],
#         f'http://localhost:8085/',
#         'predicted_normal',
#         1,
#         1,
#         value_schema=SAMPLE_SCHEMA,
# )

# # producer.send(topic_name, {'age': 34})
# # producer.send(topic_name, {'age': 9000, 'name': 'john'})

In [ ]:
# import json

# for message in consumer:
#     #print(message)
# #     movieId = message.value['movieId']
# #     userId = message.value['userId']
# #     print("movieId is "+str(movieId))
# #     print("userId is "+str(userId))
#     finalNumpyArray = np.asarray(message.value["array"])
#     result1=load_model.predict(finalNumpyArray)
# #     final_comment = {
# #     "schema": {
# #         "type": "struct",
# #         "fields": [{
# #             "type": "string",
# #             "optional": True,
# #             "field": "spanId"
# #         }, {
# #             "type": "int64",
# #             "optional": True,
# #             "field": "movieId"
# #         }, {
# #             "type": "int64",
# #             "optional": False,
# #             "field": "userId"
# #         }, {
# #             "type": "string",
# #             "optional": False,
# #             "field": "comment"
# #         }],
# #         "optional": False,
# #         "name": "comment_schema"
# #     },
# #     "payload": {
# #         "spanId": message.value['spanId'],
# #         "movieId": message.value['movieId'],
# #         "userId": message.value['userId'],
# #         "comment": message.value["comment"]
# #     }
# # }
#     final_comment = {
#         "spanId":message.value['spanId'],
#         "movieId":message.value['movieId'],
#         "userId":message.value['userId'],
#         "comment":message.value["comment"]
#     }
#     if result1>0.5:
#         final_comment['status'] = 'anomolous'
#         #final_comment['payload']['status'] = 'anomolous'
#         #producer.send('predicted_anomolous',final_comment)
#         print("ALERT :::: This can be SQL injection")
#     elif result1<=0.5:
#         final_comment['status'] = 'normal'
#         #final_comment['payload']['status'] = 'normal'
#         #producer.send('predicted_normal',final_comment)
#         #producer.produce(topic='predicted_normal', value=final_comment, on_delivery=acked)
#         #producer.produce(topic='test_predict',value=final_comment,on_delivery=delivery_report)
#         #print(producer)
#         print("It seems to be safe")
#     while True:
#         producer.poll(0.0)
#     # Serve on_delivery callbacks from previous calls to produce()
     
#         try:
#             new_comment = comment_data(spanId=final_comment['spanId'],
#                         movieId=final_comment['movieId'],
#                         userId=final_comment['userId'],
#                         comment=final_comment['comment'],
#                         status = final_comment['status'])
#             producer.produce(topic='test_predict', value=new_comment,
#                              on_delivery=delivery_report)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
          
#         except KeyboardInterrupt:
#             break
    
#         except ValueError:
#             print("Invalid input, discarding record...")
#             continue

#     print("\nFlushing records...")
#     producer.flush()
        
#     print(new_comment)

In [ ]:
# import json

# for message in consumer:
#     #print(message)
# #     movieId = message.value['movieId']
# #     userId = message.value['userId']
# #     print("movieId is "+str(movieId))
# #     print("userId is "+str(userId))
#     finalNumpyArray = np.asarray(message.value["array"])
#     result1=load_model.predict(finalNumpyArray)
#     final_comment = {
#         "spanId":message.value['spanId'],
#         "movieId":message.value['movieId'],
#         "userId":message.value['userId'],
#         "comment":message.value["comment"]
#     }
#     if result1>0.5:
#         final_comment['status'] = 'anomolous'
#         #final_comment['payload']['status'] = 'anomolous'
#         #producer.send('predicted_anomolous',final_comment)
#         print("ALERT :::: This can be SQL injection")
#     elif result1<=0.5:
#         final_comment['status'] = 'normal'
#         #final_comment['payload']['status'] = 'normal'
#         #producer.send('predicted_normal',final_comment)
#         #producer.produce(topic='predicted_normal', value=final_comment, on_delivery=acked)
#         #producer.produce(topic='test_predict',value=final_comment,on_delivery=delivery_report)
#         #print(producer)
#         print("It seems to be safe")

#     producer.poll(0.0)
#     # Serve on_delivery callbacks from previous calls to produce()
#     try:
#         new_comment = comment_data(spanId=final_comment['spanId'],
#                         movieId=final_comment['movieId'],
#                         userId=final_comment['userId'],
#                         comment=final_comment['comment'],
#                         status = final_comment['status'])
#         producer.produce(topic='test_predict', key=str(uuid4()),value=new_comment,
#                              on_delivery=delivery_report)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
          
#     except KeyboardInterrupt:
#         break
    
#     except ValueError:
#         print("Invalid input, discarding record...")
#         continue

# #     print("\nFlushing records...")
# #     producer.flush()
        
#     print(new_comment)

In [ ]:
#     "spanId":message.value['spanId'],
#         "movieId":message.value['movieId'],
#         "userId":message.value['userId'],
#         "comment":message.value["comment"]
import json
import datetime

for message in consumer:
    #print(message)
#     movieId = message.value['movieId']
#     userId = message.value['userId']
#     print("movieId is "+str(movieId))
#     print("userId is "+str(userId))
    finalNumpyArray = np.asarray(message.value["array"])
    result1=load_model.predict(finalNumpyArray)
    final_comment = {

    "schema": {
        "type": "struct",
        "optional": False,
        "version": 1,
        "fields": [
            {
                "field": "span_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "movie_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "user_id",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment",
                "type": "string",
                "optional": True
            },
            {
                "field": "status",
                "type": "string",
                "optional": True
            },
            {
                "field": "comment_date",
                "type": "string",
                "optional": True
            }
        ]
    },
    "payload": {
        "span_id": message.value['spanId'],
        "movie_id": message.value['movieId'],
        "user_id": message.value['userId'],
        "comment": message.value["comment"],
        "comment_date": message.value["comment_date"]   
    }

    }
    if result1>0.5:
#         comment_date = datetime.date.today()
#         final_comment['payload']['comment_date'] = comment_date
        final_comment['payload']['status'] = 'anomolous'
        #final_comment['payload']['status'] = 'anomolous'
        producer2.send('predicted_anomolous_1',final_comment)
        print("ALERT :::: This can be SQL injection")
    elif result1<=0.5:
#         comment_date = datetime.date.today()
#         final_comment['payload']['comment_date'] = comment_date
        final_comment['payload']['status'] = 'normal'
        #final_comment['payload']['status'] = 'normal'
        producer2.send('predicted_normal_1',final_comment)
        #producer.produce(topic='predicted_normal', value=final_comment, on_delivery=acked)
        #producer.produce(topic='test_predict',value=final_comment,on_delivery=delivery_report)
        #print(producer)
        print("It seems to be safe")

#     producer.poll(0.0)
#     # Serve on_delivery callbacks from previous calls to produce()
#     try:
#         new_comment = comment_data(spanId=final_comment['spanId'],
#                         movieId=final_comment['movieId'],
#                         userId=final_comment['userId'],
#                         comment=final_comment['comment'],
#                         status = final_comment['status'])
#         producer.produce(topic='test_predict', key=str(uuid4()),value=new_comment,
#                              on_delivery=delivery_report)
# #         user_name = input("Enter name: ")
# #         user_address = input("Enter address: ")
# #         user_favorite_number = int(input("Enter favorite number: "))
# #         user_favorite_color = input("Enter favorite color: ")
        
          
#     except KeyboardInterrupt:
#         break
    
#     except ValueError:
#         print("Invalid input, discarding record...")
#         continue

#     print("\nFlushing records...")
#     producer.flush()
        
    print(final_comment)

It seems to be safe
{'schema': {'type': 'struct', 'optional': False, 'version': 1, 'fields': [{'field': 'span_id', 'type': 'string', 'optional': True}, {'field': 'movie_id', 'type': 'string', 'optional': True}, {'field': 'user_id', 'type': 'string', 'optional': True}, {'field': 'comment', 'type': 'string', 'optional': True}, {'field': 'status', 'type': 'string', 'optional': True}, {'field': 'comment_date', 'type': 'string', 'optional': True}]}, 'payload': {'span_id': 'kviwS+9Bi20=', 'movie_id': '2', 'user_id': '0', 'comment': 'hello', 'comment_date': '2021-10-13', 'status': 'normal'}}
It seems to be safe
{'schema': {'type': 'struct', 'optional': False, 'version': 1, 'fields': [{'field': 'span_id', 'type': 'string', 'optional': True}, {'field': 'movie_id', 'type': 'string', 'optional': True}, {'field': 'user_id', 'type': 'string', 'optional': True}, {'field': 'comment', 'type': 'string', 'optional': True}, {'field': 'status', 'type': 'string', 'optional': True}, {'field': 'comment_date'